# Advent of Code 2015


See [here](http://adventofcode.com/2015/).

## Preparation

Imports and utility functions that might or might not prove useful down the line.

In [5]:
# Python 3.x
import re
import numpy as np
import math
import urllib.request
import reprlib
import operator
import string

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache, reduce
from itertools   import permutations, combinations, chain, cycle, product, islice, count, repeat, filterfalse
from heapq       import heappop, heappush
from enum        import Enum

def Input(day,strip=True):
    "Open this day's input file."
    
    filename = 'input/input{}.txt'.format(day)
    try:
        with open(filename, 'r') as f:
            text = f.read()
            if strip:
                text = text.strip()
        return text
    except FileNotFoundError:
        url = 'http://adventofcode.com/2017/day/{}/input'.format(day)
        print('input file not found. opening browser...')
        print('please save the file as "input<#day>.txt in your input folder.')
        import webbrowser
        webbrowser.open(url)

cat = ''.join
def first(iterable, default=None): return next(iter(iterable), default)
def nth(iterable, n, default=None): return next(islice(iterable, n, None), default)
def fs(*items): return frozenset(items)

def ilen(iterator): return sum(1 for _ in iterator)

def ints(text,typ=int):
    return list(map(typ,re.compile(r'[-+]?\d*[.]?\d+').findall(text)))

def shift(it, n):
    return it[n:] + it[:n]

def rot(mat, N=1, clockwise=True):
    '''rotate 2D matrix'''
    for _ in range(N):
        if clockwise:
            mat = list(zip(*mat[::-1]))
        else:
            mat = list(zip(*mat[::-1]))[::-1]
    return mat

def locate2D(m, val):
    '''locate value in 2D list'''
    for i, line in enumerate(m):
        j=-1
        try:
            j = line.index(val)
        except ValueError:
            continue
        break
    else:
        i = -1
    return (i,j)

def dist_L1(p1,p2=None):
    if p2 == None:
        p2 = repeat(0)
    return sum(abs(p2_i-p1_i) for p1_i, p2_i in zip(p1,p2))

def dist_L2(p1,p2=None):
    if p2 == None:
        p2 = repeat(0)
    return sum((p2_i-p1_i)*(p2_i-p1_i) for p1_i, p2_i in zip(p1,p2))**.5

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (x+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

from numbers import Number 
class Vector(object):
    def __init__(self,*args):
        if len(args) == 1:
            if isinstance(args,Number): self.vec = tuple(0 for _ in range(args))
            else: self.vec = tuple(*args)
        else: self.vec = tuple(args)
    def __mul__(self, other):
        if isinstance(other,Number): return Vector(other * x for x in self.vec)
        elif isinstance(other,Vector): return sum(x*y for x,y in zip(self.vec, other.vec))
        raise NotImplemented
    def __add__(self,other):
        return Vector(x+y for x,y in zip(self.vec, other.vec))
    def __sub__(self,other):
        return Vector(x-y for x,y in zip(self.vec, other.vec))
    def __iter__(self):
        return self.vec.__iter__()
    def __len__(self):
        return len(self.vec)
    def __getitem__(self, key):
        return self.vec[key]
    def __repr__(self):
        return 'Vector(' + str(self.vec)[1:-1] + ')'
    def __eq__(self, other):
        return self.vec == other.vec
    def __hash__(self):
        return hash(self.vec)

#display and debug functions
def h1(s):
    upr, brd, lwr = '▁', '█', '▔'
    return upr*(len(s)+4) + '\n'+brd+' ' + s + ' ' + brd +'\n' + lwr*(len(s)+4)

def h2(s, ch='-'):
    return s + '\n' + ch*len(s) + '\n'

h1 = lambda s: h2(s,'=')  #the other h1 is a bitch, apparently.

def print_result(day, part, text):
    print(h1('Day {} part {}: {}'.format(day, part, text)))

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    rep = reprlib.aRepr
    rep.maxother = 85
    def traced_f(*args):
        arg_strs = ', '.join(map(rep.repr, args))
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, arg_strs, result))
        return result
    return traced_f

## Day 1: Not Quite Lisp

In [7]:
def get_floor(directions):
    UP, DOWN = '()'
    return directions.count(UP) - directions.count(DOWN)


directions = Input(1)
print_result(1,1,'destination floor is ' + str(get_floor(directions)))

Day 1 part 1: destination floor is 232



In [10]:
for step in count(1):
    floor = get_floor(directions[:step])
    if floor == -1:
        break
print_result(1,2,'basement reached after {} steps'.format(step))

Day 1 part 2: basement reached after 1783 steps



## Day 2: I Was Told There Would Be No Math

Calculate how much wrapping paper is needed for a list of presents specified bei `width` x `height` x `length`.

Sample input looks like this:
```20x3x11
15x27x5
6x29x7
30x15x9
19x29x21
[...]```

Wrapping-paper-area is defined as surface area of the box (2*l*w + 2*w*h + 2*h*l) plus the area of the smallest side as slack.

In [15]:
def calc_area(l, w, h):
    a = l * w
    b = l * h
    c = w * h
    return 2 * (a + b + c) + min([a, b, c])


def parse_line(line):
    return [int(edge) for edge in line.strip().split('x')]


def solve_2_1(text):
    return sum(calc_area(*parse_line(line)) for line in text.splitlines())


assert calc_area(2, 3, 4) == 58
assert calc_area(1, 1, 10) == 43
assert calc_area(*parse_line('2x3x4')) == 58

rstr = '{} ft^2 of wrapping paper are needed.'
print_result(2, 1, rstr.format(solve_2_1(Input(2))))

Day 2 part 1: 1606483 ft^2 of wrapping paper are needed.



For part two we also need to calculate the length of a ribbon for a bow.

The ribbon required to wrap a present is the shortest distance around its sides, or the smallest perimeter of any one face. Each present also requires a bow made out of ribbon as well; the feet of ribbon required for the perfect bow is equal to the cubic feet of volume of the present. 

In [19]:
def calc_ribbon_length(l, w, h):
    a, b, c = sorted([l, w, h])
    return 2 * (a + b) + a * b * c


def solve_2_2(text):
    return sum(calc_ribbon_length(*parse_line(line)) for line in text.splitlines())


assert calc_ribbon_length(2, 3, 4) == 34
assert calc_ribbon_length(1, 1, 10) == 14

rstr2 = '{} ft of ribbon are needed.'
print_result(2, 2, rstr2.format(solve_2_2(Input(2))))

Day 2 part 2: 3842356 ft of ribbon are needed.



## Day 3: Perfectly Spherical Houses in a Vacuum

Santa moves on a 2D grid, depositing presents on every field.
He follows the following instructions:
- `<` for left
- `>` for right
- `^` for up
- `v` for down

Sample input looks like this:

```>^^v^<>v<<<v<v^>>v^^^<v<>^^><^<<^vv>>>^<<^>> [...]```

How many houses are visited at least once?

In [25]:
dirs = {'^': +1j,
        'v': -1j,
        '<': -1,
        '>': +1}


def travel(directions):
    visited = [0]
    for d in directions:
        visited.append(visited[-1] + dirs[d])
    return visited


assert len(set(travel('>'))) == 2
assert len(set(travel('^>v<'))) == 4
assert len(set(travel('^v^v^v^v^v'))) == 2

visited = travel(Input(3))
print_result(3, 1, '{} houses visited'.format(len(set(visited))))

Day 3 part 1: 2592 houses visited



In part 2 Santa and [Robo-Santa](http://i.imgur.com/Le0U3YY.jpg) take turns following the directions.

In [27]:
def split_travel(directions):
    return travel(directions[::2]) + travel(directions[1::2])

assert len(set(split_travel('^v'))) == 3
assert len(set(split_travel('^>v<'))) == 3
assert len(set(split_travel('^v^v^v^v^v'))) == 11

visited = split_travel(Input(3))
print_result(3, 2, '{} houses visited'.format(len(set(visited))))

Day 3 part 2: 2360 houses visited



## Day 4: The Ideal Stocking Stuffer